# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

One of the [key features](https://blog.langchain.dev/langchain-v02-leap-to-stability/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages and added stability.

Instead of one all encompassing Python package - LangChain has a `core` package and a number of additional supplementary packages.

We'll start by grabbing all of our LangChain related packages!

In [1]:
from rich import print as pprint
%load_ext rich

In [1]:
!pip install -qU langchain langchain-core langchain-community langchain-openai

Now we can get our Qdrant dependencies!

In [2]:
!pip install -qU qdrant-client

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-vision 3.7.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
opentelemetry-proto 1.25.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.1 which is incompatible.
proto-plus 1.23.0 requires protobuf<5.0.0dev,>=3.19.0, but you have protobuf 5.27.1 which is incompatible.


Let's finally get `tiktoken` and `pymupdf` so we can leverage them later on!

In [3]:
!pip install -qU tiktoken pymupdf

## Task 2: Set Environment Variables

We'll be leveraging OpenAI's suite of APIs - so we'll set our `OPENAI_API_KEY` `env` variable here!

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

assert os.getenv("OPENAI_API_KEY") is not None, "OpenAI API Key is not set"

## Task 3: Initialize a Simple Chain using LCEL

The first thing we'll do is familiarize ourselves with LCEL and the specific ins and outs of how we can use it!

### LLM Orchestration Tool (LangChain)

Let's dive right into [LangChain](https://www.langchain.com/)!

The first thing we want to do is create an object that lets us access OpenAI's `gpt-40` model.

In [3]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o", streaming=True)

#### ❓ Question #1:

What other models could we use, and how would the above code change?

> HINT: Check out [this page](https://platform.openai.com/docs/models/gpt-3-5-turbo) to find the answer!

#### Answer #1:

We can use: ['gpt-3.5-turbo', 'gpt-4', 'gpt-4-turbo'] and others. The above code would change by replacing `gpt-40` with the desired model.

### Prompt Template

Now, we'll set up a prompt template - more specifically a `ChatPromptTemplate`. This will let us build a prompt we can modify when we call our LLM!

In [4]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are a legendary and mythical Wizard. You speak in riddles and make obscure and pun-filled references to exotic cheeses."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages(
    [("system", system_template), ("human", human_template)]
)

### Our First Chain

Now we can set up our first chain!

A chain is simply two components that feed directly into eachother in a sequential fashion!

You'll notice that we're using the pipe operator `|` to connect our `chat_prompt` to our `llm`.

This is a simplified method of creating chains and it leverages the LangChain Expression Language, or LCEL.

You can read more about it [here](https://python.langchain.com/docs/expression_language/), but there a few features we should be aware of out of the box (taken directly from LangChain's documentation linked above):

- **Async, Batch, and Streaming Support** Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

- **Fallbacks** The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

- **Parallelism** Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

In the following code cell we have two components:

- `chat_prompt`, which is a formattable `ChatPromptTemplate` that contains a system message and a human message.

We'd like to be able to pass our own `content` (as found in our `human_template`) and then have the resulting message pair sent to our model and responded to!

In [5]:
chain = chat_prompt | openai_chat_model

Notice the pattern here:

We invoke our chain with the `dict` `{"content" : "Hello world!"}`.

It enters our chain:

`{"content" : "Hello world!"}` -> `invoke()` -> `chat_prompt`

Our `chat_prompt` returns a `PromptValue`, which is the formatted prompt. We then "pipe" the output of our `chat_prompt` into our `llm`.

`PromptValue` -> `|` -> `llm`

Our `llm` then takes the list of messages and provides an output which is return as a `str`!







In [6]:
print(chain.invoke({"content": "Hello world!"}))

content='Ah, mortal soul, I see you have summoned the great and enigmatic Wizard of Whey! To greet the world, you say? A noble endeavor, indeed. But as the moon waxes and wanes, so too do the fortunes of those who seek the arcane.\n\nRiddle me this: What once was milk but now is aged, with holes that tell stories of time well-caged?\n\nSeek the answer, and you may find a path through this labyrinth of curds and mysteries!' response_metadata={'finish_reason': 'stop'} id='run-698c5466-e9fa-4922-ab23-5120d6b87344-0'


Let's try it out with a different prompt!

In [8]:
pprint(
    chain.invoke(
        {
            "content": "Could I please have some advice on how to become a better Python Programmer?"
        }
    )
)


AIMessage(
    content='Ah, seeker of serpentine wisdom, to slither with grace through the code of the Python, you must first 
savor the intricacies of the cheese wheel of knowledge. Here are some cryptic morsels of advice for your 
quest:\n\n1. **Embrace the Camembert of Comprehension**: Dive deep into the creamy center of Python\'s official 
documentation. Its richness will nourish your understanding of core concepts and libraries.\n\n2. **Solve the 
Puzzles of Pecorino**: Challenge yourself with coding puzzles and problems from realms like LeetCode, HackerRank, 
and Project Euler. Each solved riddle will add a sharper edge to your code, like the tang of aged Pecorino.\n\n3. 
**Script in the Stilton of Simplicity**: Write code that is as straightforward and clean as a fine Stilton. Adhere 
to the Zen of Python – "Simple is better than complex."\n\n4. **Refactor with the Rind of Roquefort**: Fear not the
mold of refactoring! Regularly revisit and refine your code, much like the meticulous aging of Roquefort, to 
achieve a more robust and efficient program.\n\n5. **Pair with a Provolone Peer**: Engage in code reviews and pair 
programming. The smoky flavor of feedback from a fellow coder, like the delightful notes of Provolone, will elevate
your coding acumen.\n\n6. **Automate with the Asiago of Algorithms**: Strengthen your grasp on algorithms and data 
structures. The firm texture of Asiago symbolizes the solid foundation they provide to your coding prowess.\n\n7. 
**Explore the Gorgonzola of Git**: Master version control with Git. Its blue veins of branching and merging will 
weave a tapestry of collaborative and safe coding practices.\n\n8. **Melt into the Mozzarella of Modules**: 
Familiarize yourself with Python’s vast array of modules and libraries. Their stretchy versatility will extend your
coding capabilities manifold.\n\n9. **Attend the Feta of Forums**: Participate in communities like Stack Overflow, 
Reddit’s r/learnpython, and Python meetups. The briny wisdom of these gatherings will enrich your journey.\n\n10. 
**Craft the Cheddar of Challenges**: Set personal projects and challenges. The crumbly, yet rewarding, texture of 
Cheddar represents the satisfaction of bringing your own ideas to fruition.\n\nRemember, young coder, programming 
is like a fine cheese – it matures over time with patience and practice. May your journey through the Python 
pastures be as delightful and rewarding as a well-aged Gouda! 🧀🐍',
    response_metadata={'finish_reason': 'stop'},
    id='run-73715c1f-4458-4845-a80b-6925f1ee234e-0'
)

In [10]:
# Add streaming view
# LCEL supports streaming natively

async for chunk in chain.astream(
    {
        "content": "Could I please have some advice on how to become a better Python Programmer?"
    }
):
    print(chunk.content, end="", flush=True)


Ah, seeker of serpentine scripts, you wish to master the arcane art of the Python! To weave your spells more potently, heed these gouda-rous guidelines:

1. **Grasp the Foundation**: First, lay your hands upon the ancient texts, the tomes of basic syntax and semantics. Aged like a fine Roquefort, the fundamentals are your bedrock.

2. **Practice Daily**: As the moon turns blue with the ripening of a Stilton, so too must you code each day. Keep your skills sharp and your mind as fresh as a young Brie.

3. **Solve Puzzles**: Engage with the enigmatic scrolls of coding challenges. Websites like LeetCode and HackerRank are your labyrinths, filled with riddles to solve. It’s as satisfying as cracking the rind of a Parmigiano-Reggiano!

4. **Read the Scrolls of Others**: Study the code of those who came before you. GitHub is a vast library of open-source wisdom, as varied and rich as a platter of assorted cheeses.

5. **Write Clean Code**: Like the simplicity of a perfect Cheddar, strive for

Notice how we specifically referenced our `content` format option!

Now that we have the basics set up - let's see what we mean by "Retrieval Augmented" Generation.

## Naive RAG - Manually adding context through the Prompt Template

Let's look at how our model performs at a simple task - defining what LangChain is!

We'll redo some of our previous work to change the `system_template` to be less...verbose.

In [6]:
system_template = "You are a helpful assistant."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages(
    [("system", system_template), ("human", human_template)]
)

chat_chain = chat_prompt | openai_chat_model

pprint(chat_chain.invoke({"content": "Please define LangChain."}).content)

LangChain is a framework designed to facilitate the development of applications that are driven by language models.
This includes a variety of applications such as chatbots, Generative Question-Answering (GQA) systems, 
summarization tools, and more. LangChain is built around two key principles:

1. **Components:** LangChain provides modular components that can be used independently or in conjunction with each
other to create sophisticated applications. These components include:
   - **Model I/O:** Interfaces for interacting with language models.
   - **Data Connection:** Tools to connect language models with various data sources.
   - **Chains:** Sequences of calls to components that can be arranged to perform complex tasks.
   - **Agents:** Components that use language models to make decisions and interact dynamically with other 
components or data sources.
   - **Memory:** Mechanisms to maintain state or context over interactions.

2. **Use Cases:** LangChain offers specialized chains tailored for specific tasks such as:
   - **Question Answering:** Leveraging language models to answer questions based on provided data.
   - **Chatbots:** Creating conversational agents that can interact with users.
   - **Data Augmented Generation:** Enhancing the capabilities of language models by integrating them with external
data sources.
   - **Code Understanding:** Using language models to understand and generate code.
   - **Evaluation:** Tools to assess the performance of language model-driven applications.

By focusing on these two areas, LangChain aims to provide a robust and flexible toolkit for developers to build and
optimize their language model-based applications.

Well, that's not very good - is it!

The issue at play here is that our model was not trained on the idea of "LangChain", and so it's left with nothing but a guess - definitely not what we want the answer to be!

Let's ask another simple LangChain question!

In [14]:
pprint(
    chat_chain.invoke(
        {"content": "What is LangChain Expression Language (LECL)?"}
    ).content
)

LangChain Expression Language (LCEL) is a specialized language developed for use within the LangChain framework, 
which is designed to facilitate the building and deployment of applications that integrate with large language 
models (LLMs). LCEL enables the definition, manipulation, and execution of expressions and queries in a structured 
manner, allowing developers to create sophisticated workflows and interactions with LLMs more efficiently.

Key features of LCEL include:

1. **Structured Query Language**: LCEL provides a structured approach to querying and manipulating data, making it 
easier to interact with LLMs and other integrated systems.
2. **Interoperability**: It is designed to work seamlessly with the LangChain framework, allowing for smooth 
integration with various tools, APIs, and data sources.
3. **Flexibility**: LCEL supports a wide range of expressions and operations, enabling developers to create 
customized and complex workflows tailored to their specific needs.
4. **Ease of Use**: With its syntax and structure, LCEL aims to be user-friendly, making it accessible to 
developers who may not have extensive experience with traditional programming languages or query languages.

In summary, LCEL is a powerful tool within the LangChain ecosystem that enhances the capabilities of applications 
leveraging large language models by providing a robust and flexible language for defining and executing complex 
expressions and queries.

While it provides a confident response, that response is entirely ficticious! Not a great look, OpenAI!

However, let's see what happens when we rework our prompts - and we add the content from the docs to our prompt as context.

In [7]:
HUMAN_TEMPLATE = """
#CONTEXT:
{context}

QUERY:
{query}

Use the provide context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, response with "I don't know"
"""

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""

chat_prompt = ChatPromptTemplate.from_messages([("human", HUMAN_TEMPLATE)])

chat_chain = chat_prompt | openai_chat_model

async for chunk in chat_chain.astream(
    {"query": "What is LangChain Expression Language?", "context": CONTEXT}
):
    print(chunk.content, end="", flush=True)

LangChain Expression Language (LCEL) is a declarative way to easily compose chains together. It provides several benefits, including full support for sync, async, batch, and streaming interfaces, the ability to handle errors gracefully with fallbacks, automatic parallel execution of components that can be run in parallel, and seamless integration with LangSmith Tracing for detailed observability and debuggability of complex chains.

You'll notice that the response is much better this time. Not only does it answer the question well - but there's no trace of confabulation (hallucination) at all!

> NOTE: While RAG is an effective strategy to *help* ground LLMs, it is not nearly 100% effective. You will still need to ensure your responses are factual through some other processes

That, in essence, is the idea of RAG. We provide the model with context to answer our queries - and rely on it to translate the potentially lengthy and difficult to parse context into a natural language answer!

However, manually providing context is not scalable - and doesn't really offer any benefit.

Enter: Retrieval Pipelines.

## Task #4: Implement Naive RAG using LCEL

Now we can make a naive RAG application that will help us bridge the gap between our Pythonic implementation and a fully LangChain powered solution!

## Putting the R in RAG: Retrieval 101

In order to make our RAG system useful, we need a way to provide context that is most likely to answer our user's query to the LLM as additional context.

Let's tackle an immediate problem first: The Context Window.

All (most) LLMs have a limited context window which is typically measured in tokens. This window is an upper bound of how much stuff we can stuff in the model's input at a time.

Let's say we want to work off of a relatively large piece of source data - like the Ultimate Hitchhiker's Guide to the Galaxy. All 898 pages of it!

> NOTE: It is recommended you do not run the following cells, they are purely for demonstrative purposes.

In [19]:
context = """
EVERY HITCHHIKER'S GUIDE BOOK
"""

We can leverage our tokenizer to count the number of tokens for us!

In [20]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
len(enc.encode(context))

636144

The full set comes in at a whopping *636,144* tokens.

So, we have too much context. What can we do?

Well, the first thing that might enter your mind is: "Use a model with more context window", and we could definitely do that! However, even `gpt-4-32k` wouldn't be able to fit that whole text in the context window at once.

So, we can try splitting our document up into little pieces - that way, we can avoid providing too much context.

We have another problem now.

If we split our document up into little pieces, and we can't put all of them in the prompt. How do we decide which to include in the prompt?!

> NOTE: Content splitting/chunking strategies are an active area of research and iterative developement. There is no "one size fits all" approach to chunking/splitting at this moment. Use your best judgement to determine chunking strategies!

In order to conceptualize the following processes - let's create a toy context set!

### TextSplitting aka Chunking

We'll use the `RecursiveCharacterTextSplitter` to create our toy example.

It will split based on the following rules:

- Each chunk has a maximum size of 100 tokens
- It will try and split first on the `\n\n` character, then on the `\n`, then on the `<SPACE>` character, and finally it will split on individual tokens.

Let's implement it and see the results!

In [8]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter


def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(text)
    return len(tokens)


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    length_function=tiktoken_len,
)

In [9]:
text_splitter.__dict__


{
    '_chunk_size': 100,
    '_chunk_overlap': 0,
    '_length_function': <function tiktoken_len at 0x000001DE346342C0>,
    '_keep_separator': True,
    '_add_start_index': False,
    '_strip_whitespace': True,
    '_separators': ['\n\n', '\n', ' ', ''],
    '_is_separator_regex': False
}

In [10]:
chunks = text_splitter.split_text(CONTEXT)

In [11]:
len(chunks)

3

In [12]:
for chunk in chunks:
  pprint(chunk)
  print("----")

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several 
benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch,
and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and 
then expose it as an async streaming interface.

----


Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can 
easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in 
parallel. With LCEL syntax, any components that can be run in parallel automatically are.

----


Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important 
to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith 
for maximal observability and debuggability.

----


As is shown in our result, we've split each section into 100 token chunks - cleanly separated by `\n\n` characters!

#### 🏗️ Activity #1:

While there's nothing specifically wrong with the chunking method used above - it is a naive approach that is not sensitive to specific data formats.

Brainstorm some ideas that would split large single documents into smaller documents.

1. We can split by document sections, chapters, or paragraphs. An example of this would be splitting SEC 10-K filings by section
2. We can split by page numbers as well. This would be useful for documents that are paginated and have a consistent format
3. We can split by sentence. This would be useful for documents that are more conversational in nature

## Embeddings and Dense Vector Search

Now that we have our individual chunks, we need a system to correctly select the relevant pieces of information to answer our query.

This sounds like a perfect job for embeddings!

We'll be using OpenAI's `text-embedding-3` model as our embedding model today!

Let's load it up through LangChain.

In [13]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

#### ❓ Question #2:

What is the embedding dimension, given that we're using `text-embedding-3-small`?

> HINT: Check out the [docs](https://platform.openai.com/docs/guides/embeddings) to help you answer this question.

#### Answer #2:

The default embedding dimension for `text-embedding-3-small` is 1536, equivalent to the `ada-002` model.


### Finding the Embeddings for Our Chunks

First, let's find all our embeddings for each chunk and store them in a convenient format for later.

In [14]:
embeddings_dict = {}

for chunk in chunks:
    embeddings_dict[chunk] = embedding_model.embed_query(chunk)

In [15]:
for k, v in embeddings_dict.items():
    pprint(f"Chunk - {k}")
    print("---")
    pprint(f"Embedding - Vector of Size: {len(v)}")

Chunk - LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are 
several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch,
and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and 
then expose it as an async streaming interface.

---


Embedding - Vector of Size: 1536

Chunk - Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL 
you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in 
parallel. With LCEL syntax, any components that can be run in parallel automatically are.

---


Embedding - Vector of Size: 1536

Chunk - Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly 
important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to 
LangSmith for maximal observability and debuggability.

---


Embedding - Vector of Size: 1536

Okay, great. Let's create a query - and then embed it!

In [16]:
query = "Can LCEL help take code from the notebook to production?"

query_vector = embedding_model.embed_query(query)
print(f"Vector of Size: {len(query_vector)}")

Vector of Size: 1536


Now, let's compare it against each existing chunk's embedding by using cosine similarity.

In [17]:
import numpy as np
from numpy.linalg import norm


def cosine_similarity(vec_1, vec_2):
    return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

In [18]:
max_similarity = -float("inf")
closest_chunk = ""

for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
        closest_chunk = chunk
        max_similarity = cosine_similarity_score

print(closest_chunk)
print(max_similarity)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
0.537298487051912


And we get the expected result, which is the passage that specifically mentions prototyping in a Jupyter Notebook!

### Creating a Retriever

Now that we have an idea of how we're getting our most relevant information - let's see how we could create a pipeline that would automatically extract the closest chunk to our query and use it as context for our prompt!

First, we'll wrap the above in a helper function!

In [19]:
def retrieve_context(query, embeddings_dict, embedding_model):
    query_vector = embedding_model.embed_query(query)
    max_similarity = -float("inf")
    closest_chunk = ""

    for chunk, chunk_vector in embeddings_dict.items():
        cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

        if cosine_similarity_score > max_similarity:
            closest_chunk = chunk
            max_similarity = cosine_similarity_score

    return closest_chunk

Now, let's add it to our pipeline!

In [20]:
# Added streaming view

async def simple_rag(
    query: str, embeddings_dict: dict, embedding_model: OpenAIEmbeddings, chat_chain
):
    context = retrieve_context(query, embeddings_dict, embedding_model)

    chunks = []
    async for chunk in chat_chain.astream({"query": query, "context": context}):
        chunks.append(chunk.content)
        print(chunk.content, end="", flush=True)

    return_package = {
        "query": query,
        "response": "".join(chunks),
        "retriever_context": context,
    }

    return return_package

In [21]:
await simple_rag(
    "Can LCEL help take code from the notebook to production?",
    embeddings_dict,
    embedding_model,
    chat_chain,
)


Yes, LCEL can help take code from the notebook to production. Any chain constructed using LCEL will automatically have full support for sync, async, batch, and streaming operations. This allows for easy prototyping of a chain in a Jupyter notebook using the sync interface, and then seamlessly exposing it as an async streaming interface for production use.


{
    'query': 'Can LCEL help take code from the notebook to production?',
    'response': 'Yes, LCEL can help take code from the notebook to production. Any chain constructed using LCEL will automatically have full support for sync, async, batch, and streaming operations. This allows for easy prototyping of a chain in a Jupyter notebook using the sync interface, and then seamlessly exposing it as an async streaming interface for production use.',
    'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):\n\nAsync, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.'
}

In [22]:
await simple_rag(
    "What does LCEL do that makes it more reliable at scale?",
    embeddings_dict,
    embedding_model,
    chat_chain,
)


LCEL makes it more reliable at scale by automatically providing full support for sync, async, batch, and streaming operations. This ensures that chains constructed using LCEL can efficiently handle various types of workloads and execution modes, making them robust and scalable.


{
    'query': 'What does LCEL do that makes it more reliable at scale?',
    'response': 'LCEL makes it more reliable at scale by automatically providing full support for sync, async, batch, and streaming operations. This ensures that chains constructed using LCEL can efficiently handle various types of workloads and execution modes, making them robust and scalable.',
    'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):\n\nAsync, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.'
}

#### ❓ Question #3:

What does LCEL do that makes it more reliable at scale?

> HINT: Use your newly created `simple_rag` to help you answer this question!

#### Answer #3:

LCEL makes chains more reliable at scale by automatically providing full sync, async, batch, and streaming support. This ensures that any chain constructed using LCEL can be easily prototyped and then scaled to different execution modes without needing to rewrite the code, thus enhancing reliability and flexibility.

# 🤝 Breakout Room #2

## Task #5: Create a Simple RAG Application Using Qdrant, OpenAI, and LCEL

Now that we have a grasp on how LCEL works, and how we can use LangChain and OpenAI to interact with our data - let's step it up a notch and incorporate Qdrant!

## LangChain Powered RAG

First and foremost, LangChain provides a convenient way to store our chunks and their embeddings.

It's called a `VectorStore`!

We'll be using Drant as our `VectorStore` today. You can read more about it [here](https://qdrant.tech/documentation/).

Think of a `VectorStore` as a smart way to house your chunks and their associated embedding vectors. The implementation of the `VectorStore` also allows for smarter and more efficient search of our embedding vectors - as the method we used above would not scale well as we got into the millions of chunks.

Otherwise, the process remains relatively similar under the hood!

Let's use [Steve Jobs iPhone 2007 Presentation Introduction Speech PDF](https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf) as our data today!

### Data Collection

We'll be leveraging the `PyMUPDFLoader` to load our PDF directly from the web!

In [23]:
from langchain.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader(
    "https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf"
).load()

In [24]:
docs


[
    Document(
        page_content='© 2020 The Singju Post\nOn January 9, 2007, then Apple’s CEO Steve Jobs introduced the iPhone for the first time\nand the world of mobile devices changed forever. Here is the full keynote presentation by\nSteve Jobs….\n TRANSCRIPT: \nSteve Jobs- Apple CEO\nThis is the day I’ve been looking forward to for two and a half years.\nEvery once in a while, a revolutionary product comes along that changes everything and\nApple has been – well, first of all, one’s very fortunate if you get to work on just one of these\nin your career. Apple has been very fortunate. It’s been able to introduce a few of these into\nthe world.\n1984 – we introduced the Macintosh. It didn’t just change Apple. It changed the whole\ncomputer industry.\nIn 2001, we introduced the first iPod. And it didn’t just change the way we all listen to\nmusic, it changed the entire music industry.\nWell, today we’re introducing three revolutionary products of this class. The first one is a\

### Chunking Our Documents

Let's do the same process as we did before with our `RecursiveCharacterTextSplitter` - but this time we'll use ~200 tokens as our max chunk size!

In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=0,
    length_function=tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

In [26]:
len(split_chunks)

86

Alright, now we have 516 ~200 token long documents.

Let's verify the process worked as intended by checking our max document length.

In [27]:
max_chunk_length = 0

for chunk in split_chunks:
    max_chunk_length = max(max_chunk_length, tiktoken_len(chunk.page_content))

print(max_chunk_length)

197


Perfect! Now we can carry on to creating and storing our embeddings.

### Embeddings and Vector Storage

We'll use the `text-embedding-3-small` embedding model again - and `Qdrant` to store all our embedding vectors for easy retrieval later!

In [28]:
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="Steve Job's Speech",
)

In [29]:
query = "What are the first principles in the launch of the iPhone?"
qdrant_vectorstore.similarity_search(query)


[
    Document(
        page_content='what’s on any other phone. Now how do we do this? Well, we start with a strong foundation.\niPhone runs OSX.\nNow, why would we want to run such a sophisticated operating system on a mobile\ndevice? Well, because it’s got everything we need. It’s got multi-tasking. It’s got the best\nnetworking. It already knows how to power manage. We’ve been doing this on mobile\ncomputers for years. It’s got awesome security. And the right apps. It’s got everything from\nCocoa and the graphics and it’s got core animation built in and it’s got the audio and video\nthat OSX is famous for. It’s got all the stuff we want. And it’s built right in to iPhone. And\nthat has let us create desktop class applications and networking. Not the crippled stuff that\nyou find on most phones. This is real, desktop-class applications.',
        metadata={
            'source': 'https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.p

Now let's set up our retriever, just as we saw before, but this time using LangChain's simple `as_retriever()` method!

In [30]:
qdrant_retriever = qdrant_vectorstore.as_retriever()

#### Back to the Flow

We're ready to move to the next step!

### Setting up our RAG

We'll use the LCEL we touched on earlier to create a RAG chain.

Let's think through each part:

1. First we need to retrieve context
2. We need to pipe that context to our model
3. We need to parse that output

Let's start by setting up our prompt again, just so it's fresh in our minds!

#### 🏗️ Activity #2:

Complete the prompt so that your RAG application answers queries based on the context provided, but *does not* answer queries if the context is unrelated to the query.

In [31]:
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant that provides answers with examples. Use the provided context to answer the provided user query. 
If you do not know the answer, respond with "I don't know", and provide an example of a question that you can answer.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

#### Our RAG Chain

Notice how we have a bit of a more complex chain this time - that's because we want to return our sources with the response.

Let's break down the chain step-by-step:

1. We invoke the chain with the `question` item. Notice how we only need to provide `question` since both the retreiver and the `"question"` object depend on it.
  - We also chain our `"question"` into our `retriever`! This is what ultimately collects the context through Qdrant.
2. We assign our collected context to a `RunnablePassthrough()` from the previous object. This is going to let us simply pass it through to the next step, but still allow us to run that section of the chain.
3. We finally collect our response by chaining our prompt, which expects both a `"question"` and `"context"`, into our `llm`. We also, collect the `"context"` again so we can output it in the final response object.

The key thing to keep in mind here is that we need to pass our context through *after* we've retrieved it - to populate the object in a way that doesn't require us to call it or try and use it for something else.

In [37]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {
        "context": itemgetter("question") | qdrant_retriever,
        "question": itemgetter("question"),
    }
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)

Let's get a visual understanding of our chain!

In [38]:
!pip install -qU grandalf

In [40]:
pprint(retrieval_augmented_qa_chain.get_graph().draw_ascii())


+---------------------------------+                         
                       | Parallel<context,question>Input |                         
                       +---------------------------------+                         
                           *****                   ****                            
                        ***                            ****                        
                     ***                                   ****                    
+--------------------------------+                             **                  
| Lambda(itemgetter('question')) |                              *                  
+--------------------------------+                              *                  
                 *                                              *                  
                 *                                              *                  
                 *                                              *                  
     +----------------------+                   +--------------------------------+ 
     | VectorStoreRetriever |                   | Lambda(itemgetter('question')) | 
     +----------------------+                   +--------------------------------+ 
                           *****                   *****                           
                                ***             ***                                
                                   ***       ***                                   
                       +----------------------------------+                        
                       | Parallel<context,question>Output |                        
                       +----------------------------------+                        
                                         *                                         
                                         *                                         
                                         *                                         
                            +------------------------+                             
                            | Parallel<context>Input |                             
                            +------------------------+                             
                              ****               ****                              
                           ***                       ***                           
                         **                             **                         
     +-------------------------------+              +-------------+                
     | Lambda(itemgetter('context')) |              | Passthrough |                
     +-------------------------------+              +-------------+                
                              ****               ****                              
                                  ***         ***                                  
                                     **     **                                     
                           +-------------------------+                             
                           | Parallel<context>Output |                             
                           +-------------------------+                             
                                         *                                         
                                         *                                         
                                         *                                         
                       +---------------------------------+                         
                       | Parallel<response,context>Input |                         
                       +---------------------------------+                         
                             ***                  ****                             
                         ****                         ***                          
                       **                                ****              

Let's try another visual representation:

![image](https://i.imgur.com/Ad31AhL.png)

Let's test our chain out!

In [41]:
response = retrieval_augmented_qa_chain.invoke(
    {"question": "What is the most important thing about the iPhone?"}
)

In [43]:
pprint(response["response"].content)

The most important thing about the iPhone, according to the provided context, is its revolutionary design and 
functionality that integrates various advanced technologies and features. Steve Jobs highlights several key 
aspects:

1. **Multi-Touch Screen**: The iPhone introduced a multi-touch screen, which was a significant innovation at the 
time.
2. **Miniaturization and Custom Silicon**: The device incorporated advanced miniaturization and custom silicon, 
allowing for enhanced performance and efficiency.
3. **Power Management and OSX Integration**: It runs OSX, a sophisticated operating system that provides 
multitasking, advanced networking, and excellent power management.
4. **Featherweight Precision Enclosures and Sensors**: The iPhone features precision enclosures and advanced 
sensors.
5. **Desktop-Class Applications**: It offers desktop-class applications and networking capabilities, which were not
common in mobile devices at the time.
6. **Widescreen Video iPod**: The iPhone also functions as a widescreen video iPod, allowing users to touch and 
interact with their music.
7. **User Interface and Contact Management**: It simplifies making calls and managing contacts, syncs seamlessly 
with PCs or Macs, and has a high-resolution screen with a single Home button for ease of use.

Overall, the iPhone was presented as the ultimate digital device that combines multiple functionalities into a 
single, user-friendly package.

Example:
- **Multi-Touch Screen**: "We’ve got the multi-touch screen. A first."
- **OSX Integration**: "It’s got everything from Cocoa and the graphics and it’s got core animation built in and 
it’s got the audio and video that OSX is famous for."

If you have any other specific questions or need further details, feel free to ask!

In [44]:
for context in response["context"]:
    pprint("Context:")
    pprint(context)
    print("----")

Context:

Document(
    page_content='of the art in every facet of this design. So let me just talk a little bit about it here. We’ve 
got\nthe multi-touch screen. A first. Miniaturization, more than any we’ve done before. A lot of\ncustom silicon. 
Tremendous power management. OSX inside a mobile device. Featherweight\nprecision enclosures. Three advanced 
sensors. Desktop class applications, and of course, the\nwidescreen video iPod. We’ve been innovating like crazy 
for the last few years on this, and\nwe filed for over 200 patents for all the inventions in iPhone, and we intend 
to protect them.\nSo, a lot of high technology. I think we’re advancing the state of the art in every aspect 
of\nthis design. So iPhone is like having your life in your pocket. It’s the ultimate digital device.',
    metadata={
        'source': 
'https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf',
        'file_path': 
'https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf',
        'page': 17,
        'total_pages': 22,
        'format': 'PDF 1.4',
        'title': 'Steve Jobs iPhone 2007 Presentation (Full Transcript)',
        'author': 'Married Wildebeest',
        'subject': '',
        'keywords': '',
        'creator': '',
        'producer': 'mPDF 7.0.3',
        'creationDate': "20200415062431+00'00'",
        'modDate': "20200415062431+00'00'",
        'trapped': '',
        '_id': '85a89cab315b4f31aeddcae28bffd546',
        '_collection_name': "Steve Job's Speech"
    }
)

----


Context:

Document(
    page_content='what’s on any other phone. Now how do we do this? Well, we start with a strong 
foundation.\niPhone runs OSX.\nNow, why would we want to run such a sophisticated operating system on a 
mobile\ndevice? Well, because it’s got everything we need. It’s got multi-tasking. It’s got the best\nnetworking. 
It already knows how to power manage. We’ve been doing this on mobile\ncomputers for years. It’s got awesome 
security. And the right apps. It’s got everything from\nCocoa and the graphics and it’s got core animation built in
and it’s got the audio and video\nthat OSX is famous for. It’s got all the stuff we want. And it’s built right in 
to iPhone. And\nthat has let us create desktop class applications and networking. Not the crippled stuff that\nyou 
find on most phones. This is real, desktop-class applications.',
    metadata={
        'source': 
'https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf',
        'file_path': 
'https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf',
        'page': 2,
        'total_pages': 22,
        'format': 'PDF 1.4',
        'title': 'Steve Jobs iPhone 2007 Presentation (Full Transcript)',
        'author': 'Married Wildebeest',
        'subject': '',
        'keywords': '',
        'creator': '',
        'producer': 'mPDF 7.0.3',
        'creationDate': "20200415062431+00'00'",
        'modDate': "20200415062431+00'00'",
        'trapped': '',
        '_id': '26fe7ca350644078a1aa393be2915f57',
        '_collection_name': "Steve Job's Speech"
    }
)

...

Document(
    page_content='you think. They told me this, they said, You had me at scrolling. So, the iPhone with the\nmost 
amazing iPod ever. You can now touch your music. So that’s the iPod.\nNow, let’s take a look at a revolutionary 
phone. We want to reinvent the phone. Now, what’s\nthe killer app? The killer app is making calls! It’s amazing — 
it’s amazing how hard it is to\nmake calls on most phones. Most people actually dial them every time. Most people 
don’t\nhave very many numbers in their address book they use their recents as their address book.\nRight? How many 
of you do that? I bet more than a few. So, we want to let you use contacts\nlike never before. You can synch your 
iPhone with your PC or Mac and bring down all your\ncontacts right into your phone. So you’ve got everybody’s 
numbers with you at all times.',
    metadata={
        'source': 
'https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf',
        'file_path': 
'https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf',
        'page': 6,
        'total_pages': 22,
        'format': 'PDF 1.4',
        'title': 'Steve Jobs iPhone 2007 Presentation (Full Transcript)',
        'author': 'Married Wildebeest',
        'subject': '',
        'keywords': '',
        'creator': '',
        'producer': 'mPDF 7.0.3',
        'creationDate': "20200415062431+00'00'",
        'modDate': "20200415062431+00'00'",
        'trapped': '',
        '_id': 'bd06b35450404129a1112b2663859e19',
        '_collection_name': "Steve Job's Speech"
    }
)

----


Context:

Document(
    page_content='you set up what you want synched to your iPhone. And it’s just like an iPod. Charge and\nsynch. 
So synch with iTunes.\nThird thing I want to talk about a little is design. We’ve designed something wonderful 
for\nyour hand, just wonderful. This is what it looks like. It’s got a three-and-a-half-inch screen\non it. It’s 
really big. And, it’s the highest-resolution screen we’ve ever shipped. It’s 160\npixels per inch. Highest we’ve 
ever shipped. It’s gorgeous. And on the front, there’s only one\nbutton down there. We call it the Home button. 
Takes you Home from wherever you are.\nAnd that’s it.\nLet’s take a look at the side. It’s really thin. It’s 
thinner than any smartphone out there, at',
    metadata={
        'source': 
'https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf',
        'file_path': 
'https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf',
        'page': 3,
        'total_pages': 22,
        'format': 'PDF 1.4',
        'title': 'Steve Jobs iPhone 2007 Presentation (Full Transcript)',
        'author': 'Married Wildebeest',
        'subject': '',
        'keywords': '',
        'creator': '',
        'producer': 'mPDF 7.0.3',
        'creationDate': "20200415062431+00'00'",
        'modDate': "20200415062431+00'00'",
        'trapped': '',
        '_id': '70b8aa8b11674a93b6d3e4179be33cd9',
        '_collection_name': "Steve Job's Speech"
    }
)

----


Let's see if it can handle a query that is totally unrelated to the source documents.

In [46]:
response = retrieval_augmented_qa_chain.invoke(
    {"question": "What is the airspeed velocity of an unladen swallow?"}
)

In [47]:
pprint(response["response"].content)

I don't know the airspeed velocity of an unladen swallow.

However, I can provide information about the features introduced in the first iPhone presentation by Steve Jobs in 
2007. For example, I can tell you about the revolutionary visual voicemail feature or how the iPhone could run a 
full desktop-class email system on a mobile device. If you have any questions about those topics, feel free to ask!

In [64]:
async for chunk in retrieval_augmented_qa_chain.astream(
    {"question": "What key innovations did the iPhone introduce?"}
):
    if chunk.get("context", None):
        pass

    else:
        print(chunk["response"].content, end="", flush=True)


The iPhone introduced several key innovations that set it apart from existing mobile devices at the time. Here are some of the major innovations highlighted in Steve Jobs' 2007 presentation:

1. **Widescreen iPod with Touch Controls:**
   - The iPhone combined the functionalities of an iPod with a touchscreen interface, allowing users to touch and interact with their music.

2. **Revolutionary Mobile Phone:**
   - The iPhone aimed to reinvent the phone with a focus on ease of use, particularly in making calls. It addressed common issues such as the difficulty of dialing numbers and managing contacts. Users could synchronize contacts from their PC or Mac, ensuring they always had access to their address book.

3. **Breakthrough Internet Communications Device:**
   - The iPhone was also positioned as an advanced internet communication device, integrating these capabilities into a single device.

4. **Multi-Touch Screen:**
   - The iPhone introduced a multi-touch screen, which was a first

#### ❓ Question #4:

What key innovations did the iPhone introduce?

> HINT: Use your RAG Chain to answer this question.


#### Answer #4:

he iPhone introduced several key innovations that set it apart from existing mobile devices at the time. Here are some of the major innovations highlighted in Steve Jobs' 2007 presentation:

1. **Widescreen iPod with Touch Controls:**
   - The iPhone combined the functionalities of an iPod with a touchscreen interface, allowing users to touch and interact with their music.

2. **Revolutionary Mobile Phone:**
   - The iPhone aimed to reinvent the phone with a focus on ease of use, particularly in making calls. It addressed common issues such as the difficulty of dialing numbers and managing contacts. Users could synchronize contacts from their PC or Mac, ensuring they always had access to their address book.

3. **Breakthrough Internet Communications Device:**
   - The iPhone was also positioned as an advanced internet communication device, integrating these capabilities into a single device.

4. **Multi-Touch Screen:**
   - The iPhone introduced a multi-touch screen, which was a first for mobile devices. This allowed for more intuitive and flexible interaction compared to traditional keypads and single-touch screens.

5. **Miniaturization and Custom Silicon:**
   - The design included significant advancements in miniaturization and custom silicon, contributing to the device's sleek and compact form factor.

6. **Power Management:**
   - The iPhone featured advanced power management techniques to ensure efficient battery usage.

...
